In [21]:
# Importing necessary libraries
from datetime import datetime
from datetime import timedelta
import os  # Operating system specific functions
import time  # Time-related functions
import numpy as np  # Numerical operations library
import pandas as pd  # Data manipulation library
import seaborn as sns  # Statistical data visualization
import matplotlib.pyplot as plt  # Plotting library
import plotly.express as px  # Interactive plotting library
import plotly.graph_objects as go  # Graphing library for creating figures
from plotly.subplots import make_subplots  # Creating subplots in Plotly
plt.style.use('seaborn')  # Setting the plotting style to Seaborn
sns.set(font_scale=2)  # Setting the font scale for Seaborn plots
import warnings; 
warnings.filterwarnings('ignore')  # Suppressing warnings

In [3]:
# Define the name of the dataset
name = 'Amazon'

# Read the dataset from CSV file
data = pd.read_csv('../input/amazonstockprice/Amazon.csv')

# Convert 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Create a subset of the last 300 rows of the dataset and reset index
ds_df = data[-300:].reset_index(drop=True)

# Assign the entire dataset to another variable
ds_df2 = data

In [4]:
display(ds_df)

Date         High          Low         Open        Close     Volume  \
0   2021-01-19  3145.000000  3096.000000  3107.000000  3120.760010  3305100.0   
1   2021-01-20  3279.800049  3175.000000  3181.989990  3263.379883  5309800.0   
2   2021-01-21  3348.550049  3289.570068  3293.000000  3306.989990  4936100.0   
3   2021-01-22  3321.909912  3283.159912  3304.310059  3292.229980  2821900.0   
4   2021-01-25  3363.889893  3243.149902  3328.500000  3294.000000  3749800.0   
..         ...          ...          ...          ...          ...        ...   
295 2022-03-21  3261.679932  3191.060059  3222.419922  3229.830078  3326900.0   
296 2022-03-22  3323.340088  3233.979980  3236.110107  3297.780029  3204300.0   
297 2022-03-23  3327.399902  3253.739990  3274.100098  3268.159912  2790600.0   
298 2022-03-24  3282.370117  3201.000000  3274.989990  3272.989990  2834900.0   
299 2022-03-25  3307.370117  3247.489990  3280.000000  3282.010010  1584383.0   

       Adj Close  
0    3120.760010  
1    3263.379883  
2    3306.989990  
3    3292.229980  
4    3294.000000  
..           ...  
295  3229.830078  
296  3297.780029  
297  3268.159912  
298  3272.989990  
299  3282.010010  

[300 rows x 7 columns]

# Candlestick

In [5]:
# Create a Candlestick chart using Plotly
fig = go.Figure(go.Candlestick(
    x=ds_df['Date'],  # X-axis values from 'Date' column
    open=ds_df['Open'],  # Opening prices
    high=ds_df['High'],  # Highest prices
    low=ds_df['Low'],  # Lowest prices
    close=ds_df['Close']  # Closing prices
))

# Update the layout of the figure
fig.update_layout(
    title='Candlestick',  # Set the title of the chart
    yaxis_title='USD',  # Set the title for the y-axis
    width=800,  # Set the width of the figure
    height=500,  # Set the height of the figure
)

# Update the y-axis settings (optional, if additional settings are required)
fig.update_yaxes()

# Display the figure
fig.show()

# Exponential Moving Average

In [6]:
# Calculate Exponential Moving Average (EMA) using Pandas' ewm method
ewma = pd.Series.ewm

# Calculate 12-period Exponential Moving Average (EMA) for 'Close' prices
ds_df['rolling_ema_12'] = ds_df['Close'].ewm(min_periods=12, span=12).mean()

# Calculate 26-period Exponential Moving Average (EMA) for 'Close' prices
ds_df['rolling_ema_26'] = ds_df['Close'].ewm(min_periods=26, span=26).mean()

In [7]:
# Create a subplot figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for 'Close' prices
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['Close'], mode='lines', name='Close'),
    secondary_y=False  # Primary y-axis
)

# Add trace for 12-period EMA
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['rolling_ema_12'], mode='lines', name='EMA_12'),
    secondary_y=False  # Primary y-axis
)

# Add trace for 26-period EMA
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['rolling_ema_26'], mode='lines', name='EMA_26'),
    secondary_y=False  # Primary y-axis
)

# Update the layout of the figure
fig.update_layout(
    title='Exponential Moving Average',  # Set the title of the chart
    yaxis_title='USD',  # Set the title for the y-axis
    width=800,  # Set the width of the figure
    height=500,  # Set the height of the figure
)

# Update the y-axis settings (optional, if additional settings are required)
fig.update_yaxes()

# Display the figure
fig.show()

# MACD

In [8]:
# Calculate 12-period Exponential Moving Average (EMA) for 'Close' prices
ds_df['close_12EMA'] = ds_df['Close'].ewm(span=12).mean()

# Calculate 26-period Exponential Moving Average (EMA) for 'Close' prices
ds_df['close_26EMA'] = ds_df['Close'].ewm(span=26).mean()

# Calculate Moving Average Convergence Divergence (MACD)
ds_df['MACD'] = ds_df['close_12EMA'] - ds_df['close_26EMA']

# Calculate 9-period Exponential Moving Average (EMA) of MACD (MACD Signal line)
ds_df['MACD_Signal'] = ds_df['MACD'].ewm(span=9).mean()

In [9]:
# Create a subplot figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for 'Close' prices on secondary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['Close'], mode='lines', name='Close'),
    secondary_y=True  # Secondary y-axis
)

# Add trace for MACD on primary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['MACD'], mode='lines', name='MACD'),
    secondary_y=False  # Primary y-axis
)

# Add trace for MACD Signal on primary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['MACD_Signal'], mode='lines', name='MACD_Signal'),
    secondary_y=False  # Primary y-axis
)

# Update the layout of the figure
fig.update_layout(
    autosize=False,  # Disable autosizing
    width=800,  # Set the width of the figure
    height=400,  # Set the height of the figure
    title_text="MACD",  # Set the title of the chart
)

# Update x-axis title
fig.update_xaxes(title_text="Date")

# Update y-axis titles
fig.update_yaxes(title_text="MACD", secondary_y=False)  # Primary y-axis
fig.update_yaxes(title_text="USD", secondary_y=True)  # Secondary y-axis

# Display the figure
fig.show()

In [10]:
# Create a subplot figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for 'Close' prices on secondary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['Close'], mode='lines', name='Close'),
    secondary_y=True  # Secondary y-axis
)

# Calculate the difference between MACD and MACD Signal
macd_minus_signal = ds_df['MACD'] - ds_df['MACD_Signal']

# Add trace for MACD - MACD Signal on primary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=macd_minus_signal, mode='lines', name='MACD - MACD_Signal'),
    secondary_y=False  # Primary y-axis
)

# Update the layout of the figure
fig.update_layout(
    autosize=False,  # Disable autosizing
    width=800,  # Set the width of the figure
    height=400,  # Set the height of the figure
    title_text="MACD - MACD_Signal",  # Set the title of the chart
)

# Update x-axis title
fig.update_xaxes(title_text="Date")

# Update y-axis titles
fig.update_yaxes(title_text="MACD", secondary_y=False)  # Primary y-axis
fig.update_yaxes(title_text="USD", secondary_y=True)  # Secondary y-axis

# Display the figure
fig.show()

# RSI

In [11]:
def rsiFunc(prices, n=14):
    # Calculate differences between consecutive prices
    deltas = np.diff(prices)
    
    # Initial calculation for up and down movements
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum() / n
    down = -seed[seed < 0].sum() / n
    rs = up / down
    rsi = np.zeros_like(prices)
    
    # Calculate RSI for the first n periods
    rsi[:n] = 100. - 100. / (1. + rs)

    # Calculate RSI for subsequent periods
    for i in range(n, len(prices)):
        delta = deltas[i - 1]
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        
        # Update up and down averages using smoothing formula
        up = (up * (n - 1) + upval) / n
        down = (down * (n - 1) + downval) / n
        
        # Calculate RS and then RSI
        rs = up / down
        rsi[i] = 100. - 100. / (1. + rs)

    return rsi

In [12]:
# Calculate RSI with different periods using rsiFunc
ds_df['rsi_6'] = rsiFunc(ds_df['Close'].values, 6)
ds_df['rsi_14'] = rsiFunc(ds_df['Close'].values, 14)
ds_df['rsi_20'] = rsiFunc(ds_df['Close'].values, 20)

In [13]:
# Create a subplot figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for 'Close' prices on secondary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['Close'], mode='lines', name='Close'),
    secondary_y=True  # Secondary y-axis
)

# Add trace for RSI with period 14 on primary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['rsi_14'], mode='lines', name='RSI 14'),
    secondary_y=False  # Primary y-axis
)

# Add trace for RSI with period 20 on primary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['rsi_20'], mode='lines', name='RSI 20'),
    secondary_y=False  # Primary y-axis
)

# Update the layout of the figure
fig.update_layout(
    title='RSI',  # Set the title of the chart
    yaxis_title='USD',  # Set the title for the y-axis on the right (secondary y-axis)
    width=800,  # Set the width of the figure
    height=400,  # Set the height of the figure
)

# Update x-axis title
fig.update_xaxes(title_text="Date")

# Update y-axis titles
fig.update_yaxes(title_text="RSI", secondary_y=False)  # Primary y-axis
fig.update_yaxes(title_text="USD", secondary_y=True)  # Secondary y-axis

# Display the figure
fig.show()

# Bollinger Band

In [14]:
window = 20  # Window size for moving average
no_of_std = 2  # Number of standard deviations for Bollinger Bands

# Calculate the moving average (MA)
ds_df[f'MA_{window}MA'] = ds_df['Close'].rolling(window=window).mean()

# Calculate the standard deviation of the window
ds_df[f'MA_{window}MA_std'] = ds_df['Close'].rolling(window=window).std()

# Calculate the upper Bollinger Band (BB)
ds_df[f'MA_{window}MA_BB_high'] = ds_df[f'MA_{window}MA'] + no_of_std * ds_df[f'MA_{window}MA_std']

# Calculate the lower Bollinger Band (BB)
ds_df[f'MA_{window}MA_BB_low'] = ds_df[f'MA_{window}MA'] - no_of_std * ds_df[f'MA_{window}MA_std']

In [15]:
import plotly.graph_objects as go

# Create a new figure
fig = go.Figure()

# Update layout of the figure
fig.update_layout(
    title='Bollinger Band',  # Set title of the chart
    yaxis_title='USD',  # Set title for y-axis
    width=800,  # Set width of the figure
    height=400,  # Set height of the figure
)

# Add trace for 'Close' prices
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['Close'], mode='lines', name='Close')
)

# Add trace for Bollinger Band mean (MA)
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df[f'MA_{window}MA'], mode='lines', name='BB_mean')
)

# Add trace for upper Bollinger Band (BB_high)
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df[f'MA_{window}MA_BB_high'], mode='lines', name='BB_high')
)

# Add trace for lower Bollinger Band (BB_low)
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df[f'MA_{window}MA_BB_low'], mode='lines', name='BB_low')
)

# Display the figure
fig.show()

# Stochastic Oscillator

In [16]:
# Calculate the rolling minimum of 'Low' prices over a window of 14 periods
ds_df['Low_min'] = ds_df['Low'].rolling(window=14, center=False).min()

# Calculate the rolling maximum of 'High' prices over a window of 14 periods
ds_df['High_max'] = ds_df['High'].rolling(window=14, center=False).max()

# Calculate STOK (Stochastic Oscillator %K)
ds_df['STOK'] = ((ds_df['Close'] - ds_df['Low_min']) / (ds_df['High_max'] - ds_df['Low_min'])) * 100

# Calculate STOD (Stochastic Oscillator %D) as the 3-period moving average of STOK
ds_df['STOD'] = ds_df['STOK'].rolling(window=3, center=False).mean()

In [17]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplot figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for 'Close' prices on primary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['Close'], mode='lines', name='Close'),
    secondary_y=False  # Primary y-axis
)

# Add trace for STOK (Stochastic Oscillator %K) on secondary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['STOK'], mode='lines', name='STOK'),
    secondary_y=True  # Secondary y-axis
)

# Add trace for STOD (Stochastic Oscillator %D) on secondary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['STOD'], mode='lines', name='STOD'),
    secondary_y=True  # Secondary y-axis
)

# Update layout of the figure
fig.update_layout(
    title='Stochastic Oscillator',  # Set title of the chart
    yaxis_title='USD',  # Set y-axis title
    width=800,  # Set width of the figure
    height=400,  # Set height of the figure
)

# Display the figure
fig.show()

# Parabolic SAR

In [18]:
def psar(df, iaf=0.02, maxaf=0.2):
    length = len(df)
    dates = df['Date']
    high = df['High']
    low = df['Low']
    close = df['Close']
    psar = df['Close'].copy()
    psarbull = [None] * length
    psarbear = [None] * length
    bull = True
    af = iaf
    ep = df['Low'][0]
    hp = df['High'][0]
    lp = df['Low'][0]
    
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        
        reverse = False
        
        if bull:
            if df['Low'][i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = df['Low'][i]
                af = iaf
        else:
            if df['High'][i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = df['High'][i]
                af = iaf
        
        if not reverse:
            if bull:
                if df['High'][i] > hp:
                    hp = df['High'][i]
                    af = min(af + iaf, maxaf)
                if df['Low'][i - 1] < psar[i]:
                    psar[i] = df['Low'][i - 1]
                if df['Low'][i - 2] < psar[i]:
                    psar[i] = df['Low'][i - 2]
            else:
                if df['Low'][i] < lp:
                    lp = df['Low'][i]
                    af = min(af + iaf, maxaf)
                if df['High'][i - 1] > psar[i]:
                    psar[i] = df['High'][i - 1]
                if df['High'][i - 2] > psar[i]:
                    psar[i] = df['High'][i - 2]
        
        if bull:
            psarbull[i] = psar[i]
        else:
            psarbear[i] = psar[i]
    
    df['psar'] = psar
    df['psarbear'] = psarbear
    df['psarbull'] = psarbull
    
    return df

In [19]:
psar(ds_df)

Date         High          Low         Open        Close     Volume  \
0   2021-01-19  3145.000000  3096.000000  3107.000000  3120.760010  3305100.0   
1   2021-01-20  3279.800049  3175.000000  3181.989990  3263.379883  5309800.0   
2   2021-01-21  3348.550049  3289.570068  3293.000000  3306.989990  4936100.0   
3   2021-01-22  3321.909912  3283.159912  3304.310059  3292.229980  2821900.0   
4   2021-01-25  3363.889893  3243.149902  3328.500000  3294.000000  3749800.0   
..         ...          ...          ...          ...          ...        ...   
295 2022-03-21  3261.679932  3191.060059  3222.419922  3229.830078  3326900.0   
296 2022-03-22  3323.340088  3233.979980  3236.110107  3297.780029  3204300.0   
297 2022-03-23  3327.399902  3253.739990  3274.100098  3268.159912  2790600.0   
298 2022-03-24  3282.370117  3201.000000  3274.989990  3272.989990  2834900.0   
299 2022-03-25  3307.370117  3247.489990  3280.000000  3282.010010  1584383.0   

       Adj Close  rolling_ema_12  rolling_ema_26  close_12EMA  ...  \
0    3120.760010             NaN             NaN  3120.760010  ...   
1    3263.379883             NaN             NaN  3198.012441  ...   
2    3306.989990             NaN             NaN  3240.546404  ...   
3    3292.229980             NaN             NaN  3256.860889  ...   
4    3294.000000             NaN             NaN  3266.951453  ...   
..           ...             ...             ...          ...  ...   
295  3229.830078     3040.375978     3022.687138  3040.375978  ...   
296  3297.780029     3079.976601     3043.064389  3079.976601  ...   
297  3268.159912     3108.927880     3059.738131  3108.927880  ...   
298  3272.989990     3134.168205     3075.534565  3134.168205  ...   
299  3282.010010     3156.913098     3090.829043  3156.913098  ...   

     MA_20MA_std  MA_20MA_BB_high  MA_20MA_BB_low      Low_min     High_max  \
0            NaN              NaN             NaN          NaN          NaN   
1            NaN              NaN             NaN          NaN          NaN   
2            NaN              NaN             NaN          NaN          NaN   
3            NaN              NaN             NaN          NaN          NaN   
4            NaN              NaN             NaN          NaN          NaN   
..           ...              ...             ...          ...          ...   
295   141.585971      3261.032976     2694.689094  2671.449951  3261.679932   
296   158.651535      3309.855609     2675.249470  2671.449951  3323.340088   
297   168.284195      3347.701923     2674.565142  2671.449951  3327.399902   
298   178.201863      3379.828763     2667.021311  2671.449951  3327.399902   
299   187.133729      3408.004495     2659.469578  2671.449951  3327.399902   

          STOK       STOD         psar  psarbear     psarbull  
0          NaN        NaN  3120.760010       NaN          NaN  
1          NaN        NaN  3263.379883       NaN          NaN  
2          NaN        NaN  3096.000000       NaN  3096.000000  
3          NaN        NaN  3106.102002       NaN  3106.102002  
4          NaN        NaN  3115.799924       NaN  3115.799924  
..         ...        ...          ...       ...          ...  
295  94.603823  97.431140  2814.450177       NaN  2814.450177  
296  96.079085  96.485695  2868.117748       NaN  2868.117748  
297  90.968825  93.883911  2931.848875       NaN  2931.848875  
298  91.705173  92.917694  2995.137040       NaN  2995.137040  
299  93.080281  91.918093  3048.299098       NaN  3048.299098  

[300 rows x 27 columns]

In [20]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplot figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for psarbear on secondary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['psarbear'], mode='lines', name='psarbear'),
    secondary_y=True  # Secondary y-axis
)

# Add trace for psarbull on secondary y-axis
fig.add_trace(
    go.Scatter(x=ds_df['Date'], y=ds_df['psarbull'], mode='lines', name='psarbull'),
    secondary_y=True  # Secondary y-axis
)

# Update layout of the figure
fig.update_layout(
    title='Parabolic SAR',  # Set title of the chart
    yaxis_title='USD',  # Set y-axis title
    width=800,  # Set width of the figure
    height=400,  # Set height of the figure
)

# Display the figure
fig.show()

# Ichimoku Clouds

In [22]:
ds_df['Date']=ds_df['Date'].astype(str)
latest0 = datetime.strptime(ds_df['Date'].max(),'%Y-%m-%d').date()
dates0=[]
for i in range(1,40):
    dates0+=[(latest0+timedelta(i)).strftime('%Y-%m-%d')]
ADD=pd.DataFrame(columns=ds_df.columns)
ADD['Date']=dates0
ds_df=pd.concat([ds_df,ADD],axis=0).reset_index(drop=True)

In [23]:
ds_df['base9']=(ds_df['High'].rolling(window=9).max()+ds_df['High'].rolling(window=9).min())/2
ds_df['base26']=(ds_df['High'].rolling(window=26).max()+ds_df['High'].rolling(window=26).min())/2
ds_df['base52']=(ds_df['High'].rolling(window=52).max()+ds_df['High'].rolling(window=52).min())/2
ds_df['leading span1']=(ds_df['base9']/2+ds_df['base52']/2).shift(26)
ds_df['leading span2']=ds_df['base52'].shift(26)
ds_df['lagging span']=ds_df['Close'].shift(-26)

In [24]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=ds_df['Date'],y=ds_df['Close'],name='Close'),secondary_y=False,)
fig.add_trace(go.Scatter(x=ds_df['Date'],y=ds_df['leading span1'],name='leading span1'),secondary_y=False,)
fig.add_trace(go.Scatter(x=ds_df['Date'],y=ds_df['leading span2'],name='leading span2'),secondary_y=False,)
fig.add_trace(go.Scatter(x=ds_df['Date'],y=ds_df['lagging span'],name='lagging span'),secondary_y=False,)
fig.update_layout(autosize=False,width=800,height=400,title_text='Ichimoku Clouds')
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="USD",secondary_y=False)
fig.update_yaxes(title_text="USD",secondary_y=True)
fig.show()

# Yearly Growth (original)

In [25]:
ds_df2['Close MA20'] = ds_df2['Close'].rolling(window=20).mean()
ds_df2['Close MA20 shift year']=ds_df2['Close MA20'].shift(253)
ds_df2['Yearly Growth']=(ds_df2['Close MA20']-ds_df2['Close MA20 shift year'])*100/ds_df2['Close MA20 shift year']

In [26]:
# Initialize subplot figure with a secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for 'Close MA20' on the primary y-axis
fig.add_trace(
    go.Scatter(x=ds_df2['Date'][500:], y=ds_df2['Close MA20'][500:], mode='lines', name='Close MA20'),
    secondary_y=False  # Primary y-axis
)

# Add trace for 'Yearly Growth' on the secondary y-axis
fig.add_trace(
    go.Scatter(x=ds_df2['Date'][500:], y=ds_df2['Yearly Growth'][500:], mode='lines', name='Yearly Growth'),
    secondary_y=True  # Secondary y-axis
)

# Update layout of the figure
fig.update_layout(
    title='Close MA20 and Yearly Growth',  # Set title of the chart
    width=800,  # Set width of the figure
    height=400,  # Set height of the figure
)

# Update x-axis title
fig.update_xaxes(title_text="Date")

# Update y-axis titles for both primary and secondary y-axes
fig.update_yaxes(title_text="Close MA20 USD", secondary_y=False)
fig.update_yaxes(title_text="Yearly Growth %", secondary_y=True)

# Display the figure
fig.show()